In [1]:
import logging
import os
import sys
import math
import argparse
sys.path.append('scripts')
sys.path.append('share')
sys.path.append('python')
from collections import namedtuple
import numpy as np
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
import tensorflow
import tensorflow.keras as keras
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc

In [2]:
def doNumber(data_EC, data_Layer, data_number, data_true):

    data = {}

    outdata = np.zeros(
        (data_number.shape[0],),
        dtype=[
            ('Output_number', np.dtype(('f4', 3))),
            ('Output_number_true', 'i4')
        ]
    )

    IBL = np.logical_and(data_Layer == 0, data_EC == 0)
    Barrel = np.logical_and(data_Layer > 0, data_EC == 0)
    Endcap = data_EC != 0

    outdata['Output_number'] = data_number
    outdata['Output_number_true'] = data_true

    data['IBL'] = outdata[IBL]
    data['Barrel'] = outdata[Barrel]
    data['Endcap'] = outdata[Endcap]
    return data

In [3]:
def evaluate(data, classes):

    fpr = [[0] for i in range(len(data))]
    tpr = [[0] for i in range(len(data))]
    auc1 = [[0] for i in range(len(data))]
    
    pos, neg = classes

    for i, layer in enumerate(data):

        pos_sel = data[layer]['Output_number_true'] == pos
        neg_sel = data[layer]['Output_number_true'] == neg

        isel = np.where(
            np.logical_or(
                pos_sel,
                neg_sel,
            )
        )[0]

        fpr[i], tpr[i], _ = roc_curve(
            data[layer]['Output_number_true'][isel],
            data[layer]['Output_number'][isel][:, pos - 1],
            pos_label=pos
        )
        auc1[i] = auc(fpr[i], tpr[i])

    return fpr, tpr, auc1

In [4]:
def rocGraph(datas, labels, name, classes):

    pos, neg = classes

    fpr = [None]*len(datas)
    tpr = [None]*len(datas)
    auc1 = [None]*len(datas)

    for i, data in enumerate(datas):
        fpr[i], tpr[i], auc1[i] = evaluate(data, classes)
        
    for i, layer in enumerate(datas[0]):
        
        for j, label in enumerate(labels):
            plt.plot(fpr[j][i], tpr[j][i], label=f'{label} (AUC = {auc1[j][i]:.2f}')
        
        rand_chance = np.linspace(0, 1, 100)
        plt.plot(rand_chance,rand_chance, ':', color='grey', label='Random (AUC = 0.5)')
        plt.semilogx()
        plt.ylabel(f"Pr(Estimated: {pos}-particle | True: {pos}-particle)")
        plt.xlabel(f"Pr(Estimated: {pos}-particle | True: {neg}-particle)")
        plt.xlim([0.001, 1.05])
        plt.ylim(0,1.5)
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.figtext(0.25, 0.90,f'{layer} {pos} vs {neg}',fontweight='bold', wrap=True, horizontalalignment='right', fontsize=10)
        plt.savefig(f'output/{name}{layer}_{pos}vs{neg}_ROC.png')
        plt.close()

In [5]:
def doRocs(datas, labels, name):
    rocGraph(datas, labels, name, (3, 2))
    rocGraph(datas, labels, name, (3, 1))
    rocGraph(datas, labels, name, (2, 3))
    rocGraph(datas, labels, name, (2, 1))
    rocGraph(datas, labels, name, (1, 2))
    rocGraph(datas, labels, name, (1, 3))

In [6]:
datas = []
models = []
labels = []


#models.append("LG_apply")
#models.append("theano_kfold/theano_evensplit_kfold1")
#models.append("jz7wevensplit_applyjz7w")
#models.append("theano104_jz7wevensplit_applyjz4w")
#models.append('jz7wevensplitunshuffled_applyjz7w')
models.append("jz7w_applyjz7w")
models.append("apply_817tf")


#labels.append('LG Model on flipped jz7w')
#labels.append('even split on flipped jz7w')
#labels.append('even split shuffled on jz7w')
#labels.append('even split on unflipped jz4w')
#labels.append('even split unshuffled on jz7w')
labels.append('natural split on jz7w theano')
labels.append('natural split on jz7w tensorflow')

name = "tfvstheano/"

for model in models:
    with h5.File(f'output/{model}.h5', 'r') as data:
        data_EC = data['NN_barrelEC'][()]
        data_Layer = data['NN_layer'][()]
        data_number = data['Output_number'][()]
        data_true = data['Output_number_true'][()]

    datas.append(doNumber(data_EC, data_Layer, data_number, data_true))

'''
ECtrained = "endcap_flip_trained_apply"
Barreltrained = "barrel_trained_apply"
IBLtrained = "ibl_trained_apply"

with h5.File(f'output/{ECtrained}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataEC = doNumber(data_EC, data_Layer, data_number, data_true)

with h5.File(f'output/{Barreltrained}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataBarrel = doNumber(data_EC, data_Layer, data_number, data_true)

with h5.File(f'output/{IBLtrained}.h5', 'r') as data:
    data_EC = data['NN_barrelEC'][()]
    data_Layer = data['NN_layer'][()]
    data_number = data['Output_number'][()]
    data_true = data['Output_number_true'][()]

dataIBL = doNumber(data_EC, data_Layer, data_number, data_true)

datas.append({'IBL':dataIBL['IBL'], 'Barrel':dataBarrel['Barrel'], 'Endcap':dataEC['Endcap']})
'''

doRocs(datas, labels, name)